## Part1: Overview

- https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_1_to_4.ipynb

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
docs = loader.load()
print(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='Zennナレッジセンス - AI知見共有ブログナレッジセンス - AI知見共有ブログPublicationへの投稿🤖RAGとは？回答精度向上のためのテクニック集（基礎編）Atsushi Kadowaki2024/02/27に公開2024/03/09エンジニアチャットボットChatGPT生成 AIRAGtechはじめまして。株式会社ナレッジセンスの門脇です。普段はエンジニア兼PMとして、「社内データに基づいて回答してくれる」チャットボットをエンタープライズ企業向けに提供しています（一応、200社以上に導入実績あり）。ここで開発しているチャットボットは、ChatGPTを始めとしたLLM（Large Language Models）を活用したサービスであり、その中でもRAG（Retrieval Augmented Generative）という仕組みをガッツリ利用しています。本記事では、RAG精度向上のための知見を共有していきます。\n\n\n はじめに\n\n この記事は何\nこの記事は、LlamaIndexのAndrei氏による『A Cheat Sheet and Some Recipes For Building Advanced RAG』[1]という記事で紹介されている「RAGに関するチートシート」について、Andrei氏の許可を得て翻訳し、解説したものです。（※なお、このAndrei氏のチートシート自体、Y Gao et al.によるサーベイ論文[2]に多大な影響を受けています）\nこのチートシートを見たとき、「自分が実務でやっていることそのものだ」と感銘を受けたのを覚えています。とても有益なシートです。\n元記事で紹介されているチートシートは非常に膨大だったため、何度かに分割して紹介できればと思います。\n本記事は「基礎編」です。いわゆるAdvancedなテクニックは未だ出てきません。まずはエンジニアが、周辺のステークホルダーを説得するためのチートシートという感じに位置づけられそうです。\n\n 対象読者\n\n生成AIに取

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='Zennナレッジセンス - AI知見共有ブログナレッジセンス - AI知見共有ブログPublicationへの投稿🤖RAGとは？回答精度向上のためのテクニック集（基礎編）Atsushi Kadowaki2024/02/27に公開2024/03/09エンジニアチャットボットChatGPT生成 AIRAGtechはじめまして。株式会社ナレッジセンスの門脇です。普段はエンジニア兼PMとして、「社内データに基づいて回答してくれる」チャットボットをエンタープライズ企業向けに提供しています（一応、200社以上に導入実績あり）。ここで開発しているチャットボットは、ChatGPTを始めとしたLLM（Large Language Models）を活用したサービスであり、その中でもRAG（Retrieval Augmented Generative）という仕組みをガッツリ利用しています。本記事では、RAG精度向上のための知見を共有していきます。\n\n\n はじめに'),
 Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='この記事は何\nこの記事は、LlamaIndexのAndrei氏による『A Cheat Sheet and Some Recipes For Building Advanced RAG』[1]という記事で紹介されている「RAGに関するチートシート」について、Andrei氏の許可を得て翻訳し、解説したものです。（※なお、このAndrei氏のチートシート自体、Y Gao et al.によるサーベイ論文[2]に多大な影響を受けています）\nこのチートシートを見たとき、「自分が実務でやっていることそのものだ」と感銘を受けたのを覚えています。とても有益なシートです。\n元記事で紹介されているチートシートは非常に膨大だったため、何度かに分割して紹介できれ

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [4]:
retriever.invoke("RAGとは？")

[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGとは。なぜ必要なのか？\nRAGとは、大雑把に言うと、ファイルを参照して回答できるLLM（例えばChatGPT）を作成するための方法です。RAGがなぜ必要なのかというと、通常のLLMでは、回答の正確性を向上させるのに限界があるからです。通常のLLMでは、事実と違う内容を勝手に捏造してしまったり（ハルシネーション）、そもそも学習データに含まれていない情報（例えば公開されていない社内の文書）については、回答することはできなかったりという限界があります。\n\n\n RAGの仕組み\nこの記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。\n「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」\n非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。'),
 Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...\n\n RAGの精度向上のために必要なこと\nRAGの精度向上を試みる際に重要な要素は、以下の2点に分解できます。\n\nユーザーの質問に回答するために最も必要な（最も関連している）ドキュメント群を抽出すること\n抽出してきたドキュメント群を最大限上手く活用して、正しい回答を生成すること\n\nこの2点は、上の「RAGの仕組み」で登場

In [13]:
from langchain import hub
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
print(format_docs(retriever.invoke("RAGとは？")))

RAGとは。なぜ必要なのか？
RAGとは、大雑把に言うと、ファイルを参照して回答できるLLM（例えばChatGPT）を作成するための方法です。RAGがなぜ必要なのかというと、通常のLLMでは、回答の正確性を向上させるのに限界があるからです。通常のLLMでは、事実と違う内容を勝手に捏造してしまったり（ハルシネーション）、そもそも学習データに含まれていない情報（例えば公開されていない社内の文書）については、回答することはできなかったりという限界があります。


 RAGの仕組み
この記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。
「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」
非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。

RAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...

 RAGの精度向上のために必要なこと
RAGの精度向上を試みる際に重要な要素は、以下の2点に分解できます。

ユーザーの質問に回答するために最も必要な（最も関連している）ドキュメント群を抽出すること
抽出してきたドキュメント群を最大限上手く活用して、正しい回答を生成すること

この2点は、上の「RAGの仕組み」で登場した画像の中の黄色い枠で囲まれている部分に該当します。
※具体的な手法は、今後の記事で紹介していきます

 RAGを使ってできること

個人的には、実務でRAGを使っていて嬉しいポイントは

情報不十分なとき、回答しないことができる
独自のドキュメントに基づいて回答できる（上の画像には含まれていませんが）

事情に少し詳しいです目次はじめにこの記事は何対象読者本題RAGテクニック集（チートシート）RAGとは。なぜ必要なのか？RAGの仕組みRAGの精度向上のために必要なことRAGを使ってできることRAGの品質計測ま

In [14]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
print(rag_chain.invoke("RAGとはなんですか？"))

RAG（Retrieval Augmented Generative）とは、ファイルを参照して回答できる大規模言語モデル（LLM）を作成するための方法です。RAGは、外部データベースから関連するドキュメントを取得し、それを基に回答を生成することで、通常のLLMの限界を克服します。これにより、より正確な情報を提供できるようになります。


In [18]:
print(rag_chain.invoke("RAGのメリットは？"))

RAGのメリットは、シンプルさと始めやすさにあります。また、情報が不十分な場合には回答を控えることができ、独自のドキュメントに基づいて正確な回答を生成することが可能です。これにより、回答の精度を向上させることが期待できます。


## Part2: Indexing

In [1]:
question = "私のお気に入りのペットの種類はなんですか？"
document = "私のお気に入りのペットは猫です。"

In [2]:
import tiktoken


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


num_tokens_from_string(question, "cl100k_base")

21

In [3]:
from langchain_openai import OpenAIEmbeddings

embed = OpenAIEmbeddings()
query_result = embed.embed_query(question)
document_result = embed.embed_query(document)
len(query_result)

1536

In [4]:
import numpy as np


def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.9325834309095222


In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=0
)

splits = text_splitter.split_documents(blog_docs)

## Part3: Retrieval

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [11]:
docs = retriever.get_relevant_documents("RAGとはなんですか？")
docs

[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの仕組み\nこの記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。\n「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」\n非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。\n\nRAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...')]

## Part4: Generation

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}'), additional_kwargs={})])

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = prompt | llm
chain.invoke({"context": docs, "question": "RAGとはなんですか？"})

AIMessage(content='RAGは、ユーザーが質問すると外部データベースから関連するドキュメントを取得し、LLMに渡して回答を生成する仕組みのことです。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 397, 'total_tokens': 455, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3850b5c6-f579-4db9-83f1-c872547c0325-0', usage_metadata={'input_tokens': 397, 'output_tokens': 58, 'total_tokens': 455, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [15]:
from langchain import hub

prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("RAGとはなんですか？")

'RAGは、ユーザーが質問すると外部データベースから関連するドキュメントを取得し、LLMに渡して回答を生成する仕組みのことです。'